In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import seaborn as sns
from typing import Dict, List, Tuple
import yfinance as yf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve, classification_report
from sklearn.model_selection import KFold
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import warnings
from datetime import datetime, timedelta
from tqdm import tqdm
import gc
import os
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import traceback

warnings.filterwarnings("ignore")

NORMALIZATION_METHODS = {
    "min_max": "Min-Max normalization to range [0,1]",
    "z_score": "Z-score normalization (mean/std)",
    "median": "Divide by median normalization",
    "sigmoid": "Sigmoid function normalization",
    "tanh_estimator": "Hyperbolic tangent estimator", 
}

class NormalizationResearch:
    def __init__(
        self,
        cache_path: str = "data",
        start_date: str = "2019-01-01",
        end_date: str = "2024-12-31",
        prediction_horizon: int = 5,
        lookback_window: int = 20,
        market_index: str = "OSEBX.OL",
        fast_mode: bool = False,
        random_seed: int = 42,
        norm_methods: List[str] = None,
        baseline_method: str = "standard",
        n_folds: int = 5,
        time_series_split: bool = True,
        gap: int = 0,
        test_size: float = 0.2
    ):
        """
        Initialize the NormalizationResearch class with k-fold cross-validation.
        
        Args:
            cache_path: Path to cache data
            start_date: Start date for data analysis
            end_date: End date for data analysis
            prediction_horizon: Number of days to predict ahead
            lookback_window: Number of days to look back for LSTM
            market_index: Stock market index to analyze
            fast_mode: Whether to use a shortened time period for faster execution
            random_seed: Random seed for reproducibility
            norm_methods: List of normalization methods to evaluate
            baseline_method: Baseline normalization method
            n_folds: Number of folds for cross-validation
            time_series_split: Whether to use time series split (vs random)
            gap: Gap between train and test sets in days (to prevent data leakage)
            test_size: Proportion of data to use for testing in each fold
        """
        np.random.seed(random_seed)
        torch.manual_seed(random_seed)
        
        if fast_mode:
            print("Fast mode enabled - using shortened time period")
            start_date = (datetime.strptime(end_date, "%Y-%m-%d") - 
                         timedelta(days=365)).strftime("%Y-%m-%d")
        
        self.cache_path = cache_path
        self.start_date = start_date
        self.end_date = end_date
        self.prediction_horizon = prediction_horizon
        self.lookback_window = lookback_window
        self.market_index = market_index
        self.fast_mode = fast_mode
        self.random_seed = random_seed
        self.n_folds = n_folds
        self.time_series_split = time_series_split
        self.gap = gap
        self.test_size = test_size
        
        self.data = None
        self.features = None
        self.feature_names = []
        self.target = None
        self.dates = None
        self.results = {}
        self.baseline_perf = {}
        self.fold_results = {}
        self.actual_returns = None
        self.daily_returns = None
        self.feature_distributions = {}
        
        if norm_methods is None:
            self.norm_methods = list(NORMALIZATION_METHODS.keys())
        else:
            self.norm_methods = [method for method in norm_methods if method in NORMALIZATION_METHODS]
            if not self.norm_methods:
                raise ValueError(f"No valid normalization methods specified. Available methods: {list(NORMALIZATION_METHODS.keys())}")
        
        if baseline_method not in NORMALIZATION_METHODS:
            raise ValueError(f"Baseline method '{baseline_method}' not valid. Available methods: {list(NORMALIZATION_METHODS.keys())}")
        
        self.baseline_method = baseline_method
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
    
    def fetch_data(self) -> pd.DataFrame:
        """
        Fetches historical market data from Yahoo Finance.
        
        Returns:
            pd.DataFrame: Market data
        """
        print(f"Fetching {self.market_index} data from {self.start_date} to {self.end_date}...")
        if not os.path.exists(self.cache_path):
            os.makedirs(self.cache_path)
        else:
            cache_file = f"{self.cache_path}/{self.market_index}_{self.start_date}_{self.end_date}.parquet"
            if os.path.exists(cache_file):
                print("Data already downloaded. Loading from file...")
                self.data = pd.read_parquet(cache_file)
                if "Date" in self.data.columns:
                    self.data.set_index("Date", inplace=True)
                return self.data
    
        buffer_days = max(100, self.lookback_window * 2)
        buffer_start = (datetime.strptime(self.start_date, "%Y-%m-%d") - 
                    timedelta(days=buffer_days)).strftime("%Y-%m-%d")

        market_data = None

        market_data = yf.download(self.market_index, start=buffer_start, end=self.end_date, progress=False)
        
        required_columns = ["Close", "High", "Low", "Open", "Volume"]
        for col in required_columns:
            if col not in market_data.columns:
                print(f"Warning: {col} not found in data, using Close as fallback")
                market_data[col] = market_data["Close"] if "Close" in market_data.columns else None
        
        data = pd.DataFrame(index=market_data.index)
        data["close"] = market_data["Close"]
        data["high"] = market_data["High"]
        data["low"] = market_data["Low"]
        data["open"] = market_data["Open"]
        data["volume"] = market_data["Volume"]
        
        for col in ["close", "high", "low", "open"]:
            data[col] = data[col].ffill().bfill()

        self.data = data.loc[self.start_date:]
        print(f"Data loaded: {len(self.data)} trading days")

        self.daily_returns = self.data["close"].pct_change().dropna()

        cache_file = f"{self.cache_path}/{self.market_index}_{self.start_date}_{self.end_date}.parquet"
        self.data.to_parquet(cache_file)
        
        return self.data
    
    def engineer_features(self) -> None:
        """
        Engineers features from raw market data with strict time-awareness to prevent data leakage.
        Features are calculated using only past data points for each time step.
        """
        print("Engineering features (time-aware approach)...")
        df = self.data.copy()

        for period in [1, 2, 3, 5, 10, 21]:
            if period < len(df):
                df[f"return_{period}d"] = df["close"].pct_change(period)
        
        for period in [10, 20, 50]:
            if period < len(df):
                df[f"ma_{period}d"] = df["close"].rolling(window=period).mean()
                # Distance to moving average
                df[f"ma_dist_{period}d"] = (df["close"] / df[f"ma_{period}d"] - 1) * 100
        
        if "return_1d" in df.columns:
            for period in [10, 21]:
                if period < len(df):
                    # Historical volatility calculation
                    df[f"vol_{period}d"] = df["return_1d"].rolling(window=period).std() * np.sqrt(252)
        
        if "ma_20d" in df.columns and "ma_50d" in df.columns:
            df["ma_cross_20_50"] = (df["ma_20d"] > df["ma_50d"]).astype(int)
        
        df["high_low_ratio"] = df["high"] / df["low"]
        df["close_open_ratio"] = df["close"] / df["open"]
        
        if "volume" in df.columns and df["volume"].sum() > 0:
            df["volume_ma_10d"] = df["volume"].rolling(window=10).mean()
            df["volume_ratio"] = df["volume"] / df["volume_ma_10d"]

        if self.prediction_horizon < len(df):
            df[f"target_return_{self.prediction_horizon}d"] = df["close"].pct_change(
                self.prediction_horizon
            ).shift(-self.prediction_horizon)
            
            df["target"] = (df[f"target_return_{self.prediction_horizon}d"] > 0).astype(int)
        else:
            print(f"Warning: Not enough data points for prediction horizon of {self.prediction_horizon}")
            shorter_horizon = max(1, min(3, len(df) // 3))
            print(f"Using shorter prediction horizon: {shorter_horizon} days")
            df[f"target_return_{shorter_horizon}d"] = df["close"].pct_change(
                shorter_horizon
            ).shift(-shorter_horizon)
            
            df["target"] = (df[f"target_return_{shorter_horizon}d"] > 0).astype(int)
        
        self.feature_names = [col for col in df.columns 
                            if not col.startswith("target") 
                            and col != "close"
                            and col != "high"
                            and col != "low"
                            and col != "open"
                            and col != "volume"]
                            
        if not self.feature_names:
            print("Warning: No features generated. Creating basic features.")
            df["log_return"] = np.log(df["close"] / df["close"].shift(1))
            self.feature_names = ["log_return"]
        
        df = df.dropna()
        
        self.features = df[self.feature_names]
        self.target = df["target"]
        self.dates = df.index
        self.actual_returns = df[f"target_return_{self.prediction_horizon}d"] if f"target_return_{self.prediction_horizon}d" in df.columns else df["target_return_3d"]
        
        print(f"Features prepared: {len(self.features)} samples with {len(self.feature_names)} features")
        print(f"Class distribution: {df['target'].value_counts(normalize=True).to_dict()}")
        
        if self.fast_mode and len(self.feature_names) > 15:
            print(f"Fast mode: limiting to 15 features (from {len(self.feature_names)})")
            self.feature_names = self.feature_names[:15]
            self.features = self.features[self.feature_names]
    
    def engineer_advanced_features(self) -> None:
        """
        Adds advanced technical indicators and market regime features.
        All calculations are time-aware to prevent data leakage.
        """
        print("Adding advanced features (time-aware approach)...")
        df = self.data.copy()
        
        for period in [7, 14, 21]:
            delta = df["close"].diff()
            gain = delta.mask(delta < 0, 0)
            loss = -delta.mask(delta > 0, 0)
            avg_gain = gain.rolling(window=period).mean()
            avg_loss = loss.rolling(window=period).mean()
            rs = avg_gain / avg_loss
            df[f"rsi_{period}"] = 100 - (100 / (1 + rs))
        
        ema12 = df["close"].ewm(span=12).mean()
        ema26 = df["close"].ewm(span=26).mean()
        df["macd"] = ema12 - ema26
        df["macd_signal"] = df["macd"].ewm(span=9).mean()
        df["macd_hist"] = df["macd"] - df["macd_signal"]
        
        for period in [20]:
            mid = df["close"].rolling(window=period).mean()
            std = df["close"].rolling(window=period).std()
            df[f"bb_upper_{period}"] = mid + 2*std
            df[f"bb_lower_{period}"] = mid - 2*std
            df[f"bb_width_{period}"] = (df[f"bb_upper_{period}"] - df[f"bb_lower_{period}"]) / mid
            df[f"bb_position_{period}"] = (df["close"] - df[f"bb_lower_{period}"]) / (df[f"bb_upper_{period}"] - df[f"bb_lower_{period}"])
        
        high_low = df["high"] - df["low"]
        high_close = (df["high"] - df["close"].shift()).abs()
        low_close = (df["low"] - df["close"].shift()).abs()
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        df["atr_14"] = true_range.rolling(14).mean()
        
        tp = (df["high"] + df["low"] + df["close"]) / 3
        tp_ma = tp.rolling(window=20).mean()
        tp_md = tp.rolling(window=20).apply(lambda x: np.abs(x - x.mean()).mean())
        df["cci_20"] = (tp - tp_ma) / (0.015 * tp_md)
        
        low_min = df["low"].rolling(window=14).min()
        high_max = df["high"].rolling(window=14).max()
        df["stoch_k"] = 100 * ((df["close"] - low_min) / (high_max - low_min))
        df["stoch_d"] = df["stoch_k"].rolling(window=3).mean()
        
        obv = [0]
        for i in range(1, len(df)):
            if df["close"].iloc[i] > df["close"].iloc[i-1]:
                obv.append(obv[-1] + df["volume"].iloc[i])
            elif df["close"].iloc[i] < df["close"].iloc[i-1]:
                obv.append(obv[-1] - df["volume"].iloc[i])
            else:
                obv.append(obv[-1])
        df["obv"] = obv
        
        if "vol_10d" in df.columns:
            df["volatility_ratio"] = df["vol_10d"].pct_change(5)
            
            if "ma_dist_10d" in df.columns:
                df["trend_strength"] = abs(df["ma_dist_10d"]) / df["vol_10d"]
        
        df["day_of_week"] = pd.to_datetime(df.index).dayofweek
        df["month"] = pd.to_datetime(df.index).month
        df["quarter"] = pd.to_datetime(df.index).quarter
        df["weekday"] = pd.to_datetime(df.index).weekday < 5
        
        if "vol_10d" in df.columns and "rsi_14" in df.columns:
            df["vol_rsi_14"] = df["vol_10d"] * df["rsi_14"]
            
        if "macd" in df.columns and "rsi_14" in df.columns:
            df["macd_rsi_14"] = df["macd"] * df["rsi_14"]
        
        df["momentum_10d"] = df["close"] / df["close"].shift(10) - 1
        df["momentum_30d"] = df["close"] / df["close"].shift(30) - 1
        
        df["roc_5d"] = (df["close"] / df["close"].shift(5) - 1) * 100
        df["roc_21d"] = (df["close"] / df["close"].shift(21) - 1) * 100

        self.data = df

    def create_time_series_splits(self) -> List[Tuple[np.ndarray, np.ndarray]]:
        """
        Creates time series splits for cross-validation with proper temporal gaps.
        
        Returns:
            List[Tuple[np.ndarray, np.ndarray]]: List of (train_idx, test_idx) pairs
        """
        n_samples = len(self.features)
        indices = np.arange(n_samples)
        
        if self.time_series_split:
            print(f"Creating {self.n_folds} time series splits with gap={self.gap}")

            splits = []
            fold_size = n_samples // (self.n_folds + 1)
            
            for fold in range(self.n_folds):
                test_start = n_samples - (self.n_folds - fold) * fold_size
                train_end = test_start - self.gap - 1
                
                if train_end <= self.lookback_window:
                    print(f"Warning: Not enough data for fold {fold+1}. Skipping.")
                    continue
                
                train_indices = indices[:train_end]
                test_indices = indices[test_start:test_start + fold_size]
                
                if len(test_indices) == 0:
                    print(f"Warning: Empty test set for fold {fold+1}. Skipping.")
                    continue
                
                print(f"Fold {fold+1}: train={len(train_indices)}, test={len(test_indices)}, "
                     f"gap={test_start-train_end-1} days")
                
                splits.append((train_indices, test_indices))
        else:
            print(f"Creating {self.n_folds} random k-fold splits (no time awareness)")
            kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)
            splits = list(kf.split(indices))
        
        return splits

    def prepare_train_test_data(self, train_idx: np.ndarray, test_idx: np.ndarray) -> Tuple:
        """
        Prepares training and test data for a given fold.
        
        Args:
            train_idx: Training indices
            test_idx: Test indices
            
        Returns:
            Tuple: X_train, X_test, y_train, y_test, dates_test, returns_test
        """
        X_train = self.features.iloc[train_idx].copy()
        X_test = self.features.iloc[test_idx].copy()
        y_train = self.target.iloc[train_idx].copy()
        y_test = self.target.iloc[test_idx].copy()
        dates_test = self.dates[test_idx]
        returns_test = self.actual_returns.iloc[test_idx].copy()
        
        return X_train, X_test, y_train, y_test, dates_test, returns_test
    
    def apply_normalization(self, method: str, X_train: pd.DataFrame, X_test: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Applies normalization method to features, fitting only on training data.
        
        Args:
            method: Normalization method name
            X_train: Training features
            X_test: Test features
            
        Returns:
            Tuple: Normalized training and test features
        """
        self.feature_distributions[method] = {
            'before': {
                'train': X_train.copy(),
                'test': X_test.copy()
            },
            'after': {}
        }
            
        if method == "min_max":
            X_train_norm = X_train.copy().values
            X_test_norm = X_test.copy().values
            
            for col in range(X_train.shape[1]):
                col_min = X_train.iloc[:, col].min()
                col_max = X_train.iloc[:, col].max()
                col_range = col_max - col_min
                
                if col_range != 0:
                    X_train_norm[:, col] = (X_train.iloc[:, col].values - col_min) / col_range
                    X_test_norm[:, col] = (X_test.iloc[:, col].values - col_min) / col_range
                    X_test_norm[:, col] = np.clip(X_test_norm[:, col], 0, 1)
        
        elif method == "z_score":
            X_train_norm = X_train.copy().values
            X_test_norm = X_test.copy().values
            
            for col in range(X_train.shape[1]):
                col_mean = X_train.iloc[:, col].mean()
                col_std = X_train.iloc[:, col].std()
                
                if col_std != 0:
                    X_train_norm[:, col] = (X_train.iloc[:, col].values - col_mean) / col_std
                    X_test_norm[:, col] = (X_test.iloc[:, col].values - col_mean) / col_std
        
        elif method == "median":
            X_train_norm = X_train.copy().values
            X_test_norm = X_test.copy().values
            
            for col in range(X_train.shape[1]):
                col_median = X_train.iloc[:, col].median()
                
                if col_median != 0:
                    X_train_norm[:, col] = X_train.iloc[:, col].values / col_median
                    X_test_norm[:, col] = X_test.iloc[:, col].values / col_median
        
        elif method == "sigmoid":
            X_train_norm = X_train.copy().values
            X_test_norm = X_test.copy().values
            
            for col in range(X_train.shape[1]):
                X_train_norm[:, col] = 1 / (1 + np.exp(-X_train.iloc[:, col].values))
                X_test_norm[:, col] = 1 / (1 + np.exp(-X_test.iloc[:, col].values))
        
        elif method == "tanh_estimator":
            X_train_norm = X_train.copy().values
            X_test_norm = X_test.copy().values
            
            for col in range(X_train.shape[1]):
                col_mean = X_train.iloc[:, col].mean()
                col_std = X_train.iloc[:, col].std()
                
                if col_std != 0:
                    X_train_norm[:, col] = 0.5 * (np.tanh(0.01 * ((X_train.iloc[:, col].values - col_mean) / col_std)) + 1)
                    X_test_norm[:, col] = 0.5 * (np.tanh(0.01 * ((X_test.iloc[:, col].values - col_mean) / col_std)) + 1)
        
            X_train_norm = np.clip(X_train_norm, -10, 10)
            X_test_norm = np.clip(X_test_norm, -10, 10)
            X_train_norm = np.nan_to_num(X_train_norm)
            X_test_norm = np.nan_to_num(X_test_norm)
        
        else:
            raise ValueError(f"Unknown normalization method: {method}")
        
        self.feature_distributions[method]['after'] = {
            'train': pd.DataFrame(X_train_norm, columns=X_train.columns),
            'test': pd.DataFrame(X_test_norm, columns=X_test.columns)
        }
        
        return X_train_norm, X_test_norm
    
    def optimize_lightgbm(self, X_train: np.ndarray, y_train: np.ndarray, class_weights: Dict[int, float] = None) -> lgb.LGBMClassifier:
        """Optimize LightGBM hyperparameters using Bayesian optimization with CV.
        
        Args:
            X_train: Training features
            y_train: Training target
            class_weights: Optional class weights for imbalanced data
            
        Returns:
            lgb.LGBMClassifier: Optimized model
        """
        print("Optimizing LightGBM hyperparameters with CV...")
        
        if len(X_train) < 100:
            print(f"Warning: Not enough data for reliable optimization ({len(X_train)} samples). Using default parameters.")
            return self.train_lightgbm_default(X_train, y_train, class_weights)
        
        n_trials = 15 if self.fast_mode else 50
        n_inner_folds = 5
        
        scale_pos_weight = None
        if class_weights is not None:
            if 1 in class_weights and 0 in class_weights:
                scale_pos_weight = class_weights[1] / class_weights[0]
                
        def objective(trial):
            params = {
                "objective": "binary",
                "metric": "binary_logloss",
                "boosting_type": trial.suggest_categorical("boosting_type", ["gbdt"]),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
                "n_estimators": trial.suggest_int("n_estimators", 100, 500),
                "num_leaves": trial.suggest_int("num_leaves", 10, 50),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 5.0, log=True),
                "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 5.0, log=True),
                "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
                "random_state": self.random_seed,
                "verbosity": -1
            }
            
            if scale_pos_weight is not None:
                params["scale_pos_weight"] = scale_pos_weight

            kf = KFold(n_splits=n_inner_folds, shuffle=True, random_state=self.random_seed)
            cv_scores = []
            
            for train_idx, val_idx in kf.split(X_train):
                X_tr, X_val = X_train[train_idx], X_train[val_idx]
                y_tr, y_val = y_train[train_idx], y_train[val_idx]
                
                if len(val_idx) < 10 or len(np.unique(y_val)) < 2:
                    continue
                
                model = lgb.LGBMClassifier(**params)
                model.fit(X_tr, y_tr)
                
                preds = model.predict_proba(X_val)[:, 1]
                score = roc_auc_score(y_val, preds)
                cv_scores.append(score)

            if cv_scores:
                return np.mean(cv_scores)
            else:
                return 0.0
        
        pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=5)
        study = optuna.create_study(
            direction="maximize", 
            sampler=TPESampler(seed=self.random_seed),
            pruner=pruner
        )
        
        try:
            timeout = 300 if self.fast_mode else 900
            study.optimize(objective, n_trials=n_trials, timeout=timeout, 
                        catch=(Exception,))
            
            if len(study.trials) == 0 or study.best_trial is None:
                print("Optimization failed to complete any trials. Using default parameters.")
                return self.train_lightgbm_default(X_train, y_train, class_weights)
                
        except Exception as e:
            print(f"Optimization error: {str(e)}. Using default parameters.")
            return self.train_lightgbm_default(X_train, y_train, class_weights)
        
        try:
            if not study.best_params:
                print("No valid parameters found. Using default parameters.")
                return self.train_lightgbm_default(X_train, y_train, class_weights)
                
            print("\nBest LightGBM Parameters:")
            for key, value in study.best_params.items():
                print(f"  {key}: {value}")
            
            best_params = study.best_params
            best_params["objective"] = "binary"
            best_params["random_state"] = self.random_seed
            best_params["verbosity"] = -1
            
            if scale_pos_weight is not None:
                best_params["scale_pos_weight"] = scale_pos_weight
            
            final_model = lgb.LGBMClassifier(**best_params)
            final_model.fit(X_train, y_train)
            
            return final_model
            
        except Exception as e:
            print(f"Error building final model with best parameters: {str(e)}. Using default parameters.")
            return self.train_lightgbm_default(X_train, y_train, class_weights)
    
    def train_lightgbm_default(self, X_train: np.ndarray, y_train: np.ndarray, class_weights: Dict[int, float] = None) -> lgb.LGBMClassifier:
        """Train a LightGBM model with default parameters.
        
        Args:
            X_train: Training features
            y_train: Training target
            class_weights: Optional class weights for imbalanced data
            
        Returns:
            lgb.LGBMClassifier: Trained model
        """
        print("Training LightGBM model with default parameters...")
        
        scale_pos_weight = None
        if class_weights is not None:
            if 1 in class_weights and 0 in class_weights:
                scale_pos_weight = class_weights[1] / class_weights[0]
        
        params = {
            "objective": "binary",
            "boosting_type": "gbdt",
            "learning_rate": 0.05,
            "n_estimators": 1000,
            "max_depth": 6,
            "num_leaves": 20,
            "min_child_samples": 50,
            "min_split_gain": 0.01,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "reg_alpha": 0.5,
            "reg_lambda": 5.0,
            "random_state": self.random_seed,
            "verbosity": -1
        }
        
        if scale_pos_weight is not None:
            params["scale_pos_weight"] = scale_pos_weight
        
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        
        return model
    
    def prepare_lstm_data(self, X_train_norm: np.ndarray, X_test_norm: np.ndarray, 
                     y_train: np.ndarray, y_test: np.ndarray, batch_size: int = 32) -> Tuple:
        """
        Prepares sequential data for LSTM model with time-aware approach.
        Adapts lookback window if necessary to ensure data availability.
        
        Args:
            X_train_norm: Normalized training features
            X_test_norm: Normalized test features
            y_train: Training target
            y_test: Test target
            batch_size: Batch size for DataLoader
            
        Returns:
            Tuple: train_loader, X_test_lstm, y_test_lstm, lookback
        """
        max_possible_lookback = min(len(X_train_norm), len(X_test_norm)) - 5
        
        if max_possible_lookback < 3:
            raise ValueError(f"Insufficient data for LSTM sequences. Need at least 8 samples, got {len(X_train_norm)} train, {len(X_test_norm)} test")
        
        lookback = min(self.lookback_window, max_possible_lookback)
        if lookback < self.lookback_window:
            print(f"Warning: Reduced lookback window from {self.lookback_window} to {lookback} due to limited data")
        
        X_train_seq = []
        y_train_seq = []
        
        for i in range(lookback, len(X_train_norm)):
            X_train_seq.append(X_train_norm[i-lookback:i])
            y_train_seq.append(y_train.iloc[i])
        
        X_test_seq = []
        y_test_seq = []
        
        for i in range(lookback, len(X_test_norm)):
            X_test_seq.append(X_test_norm[i-lookback:i])
            y_test_seq.append(y_test.iloc[i])
        
        if not X_train_seq or not X_test_seq:
            raise ValueError(f"Could not create any valid sequences with lookback={lookback}")
        
        X_train_seq = np.array(X_train_seq)
        y_train_seq = np.array(y_train_seq)
        X_test_seq = np.array(X_test_seq)
        y_test_seq = np.array(y_test_seq)
        
        print(f"Created LSTM sequences with lookback={lookback}: {len(X_train_seq)} train, {len(X_test_seq)} test")
        
        X_train_tensor = torch.FloatTensor(X_train_seq)
        y_train_tensor = torch.FloatTensor(y_train_seq).unsqueeze(1)
        
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        batch_size = min(batch_size, len(X_train_tensor))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        return train_loader, torch.FloatTensor(X_test_seq), torch.FloatTensor(y_test_seq), lookback

    class BidirectionalLSTM(nn.Module):
        """
        Bidirectional LSTM neural network model with dropout for regularization.
        """
        def __init__(self, input_dim: int, hidden_dim: int = 64, dropout: float = 0.3):
            super().__init__()
            
            self.lstm = nn.LSTM(
                input_dim, 
                hidden_dim, 
                batch_first=True, 
                bidirectional=True,
                dropout=dropout
            )
            
            self.fc = nn.Sequential(
                nn.Linear(hidden_dim * 2, 32),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(32, 1),
                nn.Sigmoid()
            )
            
        def forward(self, x: torch.Tensor) -> torch.Tensor:
            lstm_out, _ = self.lstm(x)
            lstm_out = lstm_out[:, -1]
            return self.fc(lstm_out)
    
    def train_lstm(self, train_loader: DataLoader, input_dim: int, lstm_params: Dict = None) -> nn.Module:
        """
        Trains an LSTM model with early stopping to prevent overfitting.
        
        Args:
            train_loader: DataLoader with training data
            input_dim: Number of input features
            lstm_params: Optional hyperparameters from optimization
            
        Returns:
            nn.Module: Trained LSTM model
        """
        print("Training LSTM model...")
        
        if lstm_params is None:
            hidden_dim = 64
            dropout = 0.3
            learning_rate = 0.001
            weight_decay = 1e-5
        else:
            hidden_dim = lstm_params.get("hidden_dim", 64)
            dropout = lstm_params.get("dropout", 0.3)
            learning_rate = lstm_params.get("learning_rate", 0.001)
            weight_decay = lstm_params.get("weight_decay", 1e-5)
        
        model = self.BidirectionalLSTM(input_dim, hidden_dim=hidden_dim, dropout=dropout).to(self.device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        
        num_epochs = 5 if self.fast_mode else 100
        patience = 10
        best_loss = float('inf')
        patience_counter = 0
        
        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            
            for inputs, labels in train_loader:
                inputs = inputs.to(self.device)
                labels = labels.to(self.device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                
                total_loss += loss.item()
            
            avg_loss = total_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                patience_counter = 0
                if not self.fast_mode:
                    best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1
                if patience_counter >= patience and not self.fast_mode:
                    print(f"Early stopping at epoch {epoch+1}")
                    model.load_state_dict(best_model_state)
                    break
        
        return model
    
    def optimize_lstm(self, X_train_norm: np.ndarray, y_train: np.ndarray, lookback: int) -> Dict:
        """Optimize LSTM hyperparameters using Bayesian optimization with CV.
        
        Args:
            X_train_norm: Normalized training features
            y_train: Training target
            lookback: Lookback window size
            
        Returns:
            Dict: Optimized hyperparameters
        """
        print("Optimizing LSTM hyperparameters with CV...")
        
        if len(X_train_norm) < 100:
            print(f"Warning: Not enough data for reliable LSTM optimization ({len(X_train_norm)} samples). Using default parameters.")
            return {
                "hidden_dim": 64,
                "dropout": 0.3, 
                "learning_rate": 0.001,
                "batch_size": 32,
                "weight_decay": 1e-5
            }
        
        n_trials = 10 if self.fast_mode else 100
        n_inner_folds = 3
        
        X_train_seq = []
        y_train_seq = []
        
        for i in range(lookback, len(X_train_norm)):
            X_train_seq.append(X_train_norm[i-lookback:i])
            y_train_seq.append(y_train.iloc[i])
        
        X_train_seq = np.array(X_train_seq)
        y_train_seq = np.array(y_train_seq)
        
        X_train_tensor = torch.FloatTensor(X_train_seq)
        y_train_tensor = torch.FloatTensor(y_train_seq).unsqueeze(1)
        
        num_samples = len(X_train_tensor)
        feature_dim = X_train_norm.shape[1]
        
        def objective(trial):
            hidden_dim = trial.suggest_int("hidden_dim", 32, 128)
            dropout = trial.suggest_float("dropout", 0.1, 0.5)
            
            learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
            batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
            weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True)
            
            kf = KFold(n_splits=n_inner_folds, shuffle=True, random_state=self.random_seed)
            cv_scores = []
            
            try:
                for train_idx, val_idx in kf.split(range(num_samples)):
                    X_tr, X_val = X_train_tensor[train_idx], X_train_tensor[val_idx]
                    y_tr, y_val = y_train_tensor[train_idx], y_train_tensor[val_idx]
                    
                    if len(val_idx) < 10 or len(torch.unique(y_val)) < 2:
                        continue
                    
                    train_dataset = TensorDataset(X_tr, y_tr)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    
                    model = self.BidirectionalLSTM(
                        input_dim=feature_dim,
                        hidden_dim=hidden_dim,
                        dropout=dropout
                    ).to(self.device)
                    
                    criterion = nn.BCELoss()
                    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
                    
                    epochs = 3 if self.fast_mode else 100
                    for epoch in range(epochs):
                        model.train()
                        for inputs, labels in train_loader:
                            inputs = inputs.to(self.device)
                            labels = labels.to(self.device)
                            
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            
                            loss.backward()
                            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                            optimizer.step()
                    
                    model.eval()
                    with torch.no_grad():
                        X_val = X_val.to(self.device)
                        y_val = y_val.to(self.device)
                        
                        outputs = model(X_val)
                        probs = outputs.cpu().numpy().flatten()
                        y_true = y_val.cpu().numpy().flatten()
                        
                        try:
                            score = roc_auc_score(y_true, probs)
                            cv_scores.append(score)
                        except:
                            continue
                
                if cv_scores:
                    return np.mean(cv_scores)
                else:
                    return 0.0
                
            except Exception as e:
                print(f"LSTM trial failed: {str(e)}")
                return 0.0
        
        pruner = MedianPruner(n_startup_trials=5, n_warmup_steps=5)
        study = optuna.create_study(
            direction="maximize", 
            sampler=TPESampler(seed=self.random_seed),
            pruner=pruner
        )

        lstm_default_params = {
            "hidden_dim": 64,
            "dropout": 0.3, 
            "learning_rate": 0.001,
            "batch_size": 32,
            "weight_decay": 1e-5
        }

        try:
            timeout = 300 if self.fast_mode else 900
            study.optimize(objective, n_trials=n_trials, timeout=timeout, 
                        catch=(Exception,))
            
            if len(study.trials) == 0 or study.best_trial is None:
                print("LSTM optimization failed to complete any trials. Using default parameters.")
                return lstm_default_params
                    
        except Exception as e:
            print(f"LSTM optimization error: {str(e)}. Using default parameters.")
            return lstm_default_params
        
        try:
            if not study.best_params:
                print("No valid parameters found for LSTM. Using default parameters.")
                return lstm_default_params
                    
            print("\nBest LSTM Parameters:")
            for key, value in study.best_params.items():
                print(f"  {key}: {value}")
            
            return study.best_params
                
        except Exception as e:
            print(f"Error building LSTM model with best parameters: {str(e)}. Using default parameters.")
            return lstm_default_params
    
    def evaluate_models(self, lgb_model: lgb.LGBMClassifier, lstm_model: nn.Module, 
                      X_test: np.ndarray, y_test: np.ndarray, 
                      X_test_lstm: torch.Tensor, y_test_lstm: torch.Tensor,
                      actual_returns: pd.Series, dates_test: pd.DatetimeIndex) -> Dict:
        """
        Evaluates trained models on test data with comprehensive metrics.
        Handles potential length mismatches between LSTM and regular test data.
        
        Args:
            lgb_model: Trained LightGBM model
            lstm_model: Trained LSTM model
            X_test: Test features for LightGBM
            y_test: Test target for LightGBM
            X_test_lstm: Test features for LSTM
            y_test_lstm: Test target for LSTM
            actual_returns: Actual returns for the test period
            dates_test: Dates for the test data
            
        Returns:
            Dict: Evaluation metrics
        """
        print("Evaluating models...")
        
        lgb_preds = lgb_model.predict(X_test)
        lgb_preds_proba = lgb_model.predict_proba(X_test)[:, 1]
        
        predictions_df = pd.DataFrame({
            'date': dates_test,
            'actual_return': actual_returns,
            'actual_direction': y_test,
            'lgb_pred': lgb_preds,
            'lgb_pred_proba': lgb_preds_proba
        })
        predictions_df.set_index('date', inplace=True)
        
        metrics = {}
        
        metrics["lightgbm"] = {
            "accuracy": accuracy_score(y_test, lgb_preds),
            "f1": f1_score(y_test, lgb_preds),
            "precision": precision_score(y_test, lgb_preds),
            "recall": recall_score(y_test, lgb_preds),
            "roc_auc": roc_auc_score(y_test, lgb_preds_proba) if len(np.unique(y_test)) > 1 else 0.5
        }
        
        cm = confusion_matrix(y_test, lgb_preds)
        metrics["lightgbm"]["confusion_matrix"] = cm
        
        metrics["lightgbm"]["classification_report"] = classification_report(y_test, lgb_preds, output_dict=True)
        
        if lstm_model is not None and len(X_test_lstm) > 0 and len(y_test_lstm) > 0:
            try:
                if len(X_test_lstm) != len(y_test_lstm):
                    print(f"Warning: LSTM test data length mismatch - X: {len(X_test_lstm)}, y: {len(y_test_lstm)}")
                    print("Using minimum length for evaluation")
                    min_len = min(len(X_test_lstm), len(y_test_lstm))
                    X_test_lstm = X_test_lstm[:min_len]
                    y_test_lstm = y_test_lstm[:min_len]
                
                lstm_model.eval()
                with torch.no_grad():
                    lstm_outputs = lstm_model(X_test_lstm.to(self.device))
                    lstm_preds_proba = lstm_outputs.cpu().numpy().flatten()
                    
                    if len(lstm_preds_proba) != len(y_test_lstm):
                        print(f"Warning: LSTM output length mismatch - pred: {len(lstm_preds_proba)}, y: {len(y_test_lstm)}")
                        min_len = min(len(lstm_preds_proba), len(y_test_lstm))
                        lstm_preds_proba = lstm_preds_proba[:min_len]
                        y_test_lstm = y_test_lstm[:min_len]
                    
                    lstm_preds = (lstm_preds_proba > 0.5).astype(int)
                
                y_test_lstm_np = y_test_lstm.numpy()
                
                metrics["lstm"] = {
                    "accuracy": accuracy_score(y_test_lstm_np, lstm_preds),
                    "f1": f1_score(y_test_lstm_np, lstm_preds),
                    "precision": precision_score(y_test_lstm_np, lstm_preds),
                    "recall": recall_score(y_test_lstm_np, lstm_preds),
                    "roc_auc": roc_auc_score(y_test_lstm_np, lstm_preds_proba) if len(np.unique(y_test_lstm_np)) > 1 else 0.5
                }
                
                cm_lstm = confusion_matrix(y_test_lstm_np, lstm_preds)
                metrics["lstm"]["confusion_matrix"] = cm_lstm
                
                diff = len(predictions_df) - len(lstm_preds)
                
                padded_lstm_preds = np.full(len(predictions_df), np.nan)
                padded_lstm_probs = np.full(len(predictions_df), np.nan)
                
                padded_lstm_preds[diff:] = lstm_preds
                padded_lstm_probs[diff:] = lstm_preds_proba
                
                predictions_df['lstm_pred'] = padded_lstm_preds
                predictions_df['lstm_pred_proba'] = padded_lstm_probs
                
                print(f"Successfully aligned predictions with {diff} NaN padding values")
                
                print(f"LSTM - Accuracy: {metrics['lstm']['accuracy']:.4f}, "
                    f"F1: {metrics['lstm']['f1']:.4f}, "
                    f"ROC AUC: {metrics['lstm']['roc_auc']:.4f}")
                    
            except Exception as e:
                print(f"Error evaluating LSTM model: {str(e)}")
                traceback.print_exc()
        
        print(f"LightGBM - Accuracy: {metrics['lightgbm']['accuracy']:.4f}, "
             f"F1: {metrics['lightgbm']['f1']:.4f}, "
             f"ROC AUC: {metrics['lightgbm']['roc_auc']:.4f}")
        
        metrics["predictions"] = predictions_df
        
        return metrics
    
    def run_kfold_cv(self) -> Dict:
        """
        Performs k-fold cross-validation on the dataset for all normalization methods.
        Uses time series splits to prevent data leakage.
        Optimizes both LightGBM and LSTM models for the baseline method.
        
        Returns:
            Dict: Results for each normalization method and fold
        """
        print(f"\nRunning {self.n_folds}-fold cross-validation")
        
        splits = self.create_time_series_splits()
        
        methods_to_evaluate = self.norm_methods.copy()
        if self.baseline_method not in methods_to_evaluate:
            methods_to_evaluate.append(self.baseline_method)

        self.fold_results = {
            method: {
                f"fold_{i+1}": {} for i in range(len(splits))
            } for method in methods_to_evaluate
        }
        
        print("\nOptimizing LightGBM and LSTM parameters for baseline method...")
        optimized_lgb_params = {}
        optimized_lstm_params = {}
        
        for fold_idx, (train_idx, test_idx) in enumerate(splits):
            X_train, X_test, y_train, y_test, dates_test, returns_test = self.prepare_train_test_data(
                train_idx, test_idx
            )
            
            if len(X_train) < 30 or len(X_test) < 10:
                continue
                
            class_counts = pd.Series(y_train).value_counts()
            if 0 in class_counts and 1 in class_counts:
                class_weights = {0: 1.0, 1: class_counts[0] / class_counts[1]}
            else:
                class_weights = None
            
            X_train_norm, X_test_norm = self.apply_normalization(self.baseline_method, X_train, X_test)
            
            if not self.fast_mode:
                try:
                    print(f"\nOptimizing LightGBM for fold {fold_idx+1} using {self.baseline_method} normalization")
                    lgb_model = self.optimize_lightgbm(X_train_norm, y_train.values, class_weights)
                    lgb_params = {
                        key: value for key, value in lgb_model.get_params().items() 
                        if key in ["learning_rate", "n_estimators", "num_leaves", "max_depth", 
                                "subsample", "colsample_bytree", "reg_alpha", "reg_lambda", 
                                "min_child_samples"]
                    }
                    optimized_lgb_params[f"fold_{fold_idx+1}"] = lgb_params
                except Exception as e:
                    print(f"LightGBM optimization failed for fold {fold_idx+1}: {str(e)}")
                    optimized_lgb_params[f"fold_{fold_idx+1}"] = None
                
                try:
                    print(f"\nOptimizing LSTM for fold {fold_idx+1} using {self.baseline_method} normalization")
                    _, _, _, lookback = self.prepare_lstm_data(X_train_norm, X_test_norm, y_train, y_test)
                    
                    lstm_params = self.optimize_lstm(X_train_norm, y_train, lookback)
                    optimized_lstm_params[f"fold_{fold_idx+1}"] = lstm_params
                except Exception as e:
                    print(f"LSTM optimization failed for fold {fold_idx+1}: {str(e)}")
                    optimized_lstm_params[f"fold_{fold_idx+1}"] = None
            else:
                optimized_lgb_params[f"fold_{fold_idx+1}"] = None
                optimized_lstm_params[f"fold_{fold_idx+1}"] = None
        
        print("\nStarting k-fold evaluation for all normalization methods")
        
        for fold_idx, (train_idx, test_idx) in enumerate(splits):
            print(f"\nProcessing fold {fold_idx+1}/{len(splits)}")
            
            X_train, X_test, y_train, y_test, dates_test, returns_test = self.prepare_train_test_data(
                train_idx, test_idx
            )
            
            if len(X_train) < 30 or len(X_test) < 10:
                print(f"Warning: Fold {fold_idx+1} has insufficient data (train: {len(X_train)}, test: {len(X_test)})")
                print("Skipping this fold")
                continue
            
            class_counts = pd.Series(y_train).value_counts()
            print(f"Class distribution in fold {fold_idx+1}: {class_counts.to_dict()}")
            
            if 0 in class_counts and 1 in class_counts:
                class_weights = {0: 1.0, 1: class_counts[0] / class_counts[1]}
            else:
                class_weights = None
            
            lgb_params = optimized_lgb_params.get(f"fold_{fold_idx+1}")
            lstm_params = optimized_lstm_params.get(f"fold_{fold_idx+1}")
            
            for method in tqdm(methods_to_evaluate, desc=f"Fold {fold_idx+1} - Evaluating methods"):
                print(f"\nTesting normalization method: {method} (Fold {fold_idx+1})")
                
                try:
                    X_train_norm, X_test_norm = self.apply_normalization(method, X_train, X_test)
                    
                    if lgb_params is not None:
                        params = lgb_params.copy()
                        params["random_state"] = self.random_seed
                        params["verbosity"] = -1
                        params["objective"] = "binary"
                        
                        if class_weights is not None and 1 in class_weights and 0 in class_weights:
                            params["scale_pos_weight"] = class_weights[1] / class_weights[0]
                        
                        lgb_model = lgb.LGBMClassifier(**params)
                        lgb_model.fit(X_train_norm, y_train.values)
                    else:
                        lgb_model = self.train_lightgbm_default(X_train_norm, y_train.values, class_weights)
                    
                    lstm_model = None
                    model_metrics = {}
                    
                    try:
                        batch_size = lstm_params.get("batch_size", 32) if lstm_params else 32
                        
                        train_loader, X_test_lstm, y_test_lstm, _ = self.prepare_lstm_data(
                            X_train_norm, X_test_norm, y_train, y_test, batch_size
                        )
                        
                        lstm_model = self.train_lstm(train_loader, X_train.shape[1], lstm_params)
                        
                        model_metrics = self.evaluate_models(
                            lgb_model, lstm_model, 
                            X_test_norm, y_test, 
                            X_test_lstm, y_test_lstm,
                            returns_test, dates_test
                        )
                    except Exception as e:
                        print(f"LSTM training/evaluation failed: {str(e)}")
                        print("Evaluating only LightGBM model")
                        
                        lgb_preds = lgb_model.predict(X_test_norm)
                        lgb_preds_proba = lgb_model.predict_proba(X_test_norm)[:, 1]
                        
                        model_metrics = {
                            "lightgbm": {
                                "accuracy": accuracy_score(y_test, lgb_preds),
                                "f1": f1_score(y_test, lgb_preds),
                                "precision": precision_score(y_test, lgb_preds),
                                "recall": recall_score(y_test, lgb_preds),
                                "roc_auc": roc_auc_score(y_test, lgb_preds_proba) if len(np.unique(y_test)) > 1 else 0.5,
                                "confusion_matrix": confusion_matrix(y_test, lgb_preds),
                                "classification_report": classification_report(y_test, lgb_preds, output_dict=True)
                            }
                        }
                        
                        predictions_df = pd.DataFrame({
                            'date': dates_test,
                            'actual_return': returns_test,
                            'actual_direction': y_test,
                            'lgb_pred': lgb_preds,
                            'lgb_pred_proba': lgb_preds_proba
                        })
                        predictions_df.set_index('date', inplace=True)
                        
                        model_metrics["predictions"] = predictions_df
                    
                    self.fold_results[method][f"fold_{fold_idx+1}"] = model_metrics
                    
                except Exception as e:
                    print(f"Error evaluating {method} (Fold {fold_idx+1}): {str(e)}")
                    self.fold_results[method][f"fold_{fold_idx+1}"] = {"error": str(e)}
                
                if "lgb_model" in locals():
                    del lgb_model
                if "lstm_model" in locals():
                    del lstm_model
                if "train_loader" in locals():
                    del train_loader
                if "X_test_lstm" in locals():
                    del X_test_lstm
                if "y_test_lstm" in locals():
                    del y_test_lstm
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
        
        self.process_fold_results()
        
        return self.results
    
    def process_fold_results(self) -> None:
        """
        Process fold results to calculate average metrics and standard deviations.
        Includes additional financial and predictive performance metrics.
        """
        print("\nProcessing cross-validation results...")
        
        self.results = {}
        
        for method in self.fold_results:
            method_results = {}
            
            for model_name in ["lightgbm", "lstm"]:
                model_metrics = {}
                
                for metric in ["accuracy", "f1", "precision", "recall", "roc_auc"]:
                    metric_values = []
                    
                    for fold in range(len(self.fold_results[method])):
                        fold_name = f"fold_{fold+1}"
                        if (fold_name in self.fold_results[method] and
                            model_name in self.fold_results[method][fold_name] and
                            metric in self.fold_results[method][fold_name][model_name]):
                            metric_values.append(self.fold_results[method][fold_name][model_name][metric])
                    
                    if metric_values:
                        model_metrics[metric] = np.mean(metric_values)
                        model_metrics[f"{metric}_std"] = np.std(metric_values)
                        model_metrics[f"{metric}_values"] = metric_values
                
                if model_metrics:
                    method_results[model_name] = model_metrics
            
            self.results[method] = method_results
            
            if method == self.baseline_method:
                self.baseline_perf = method_results
    
    def run_normalization_comparison(self) -> Dict:
        """
        Runs the complete normalization comparison experiment with k-fold CV.
        
        Returns:
            Dict: Comparison results
        """
        print("\n" + "="*50)
        print(f"NORMALIZATION TECHNIQUES COMPARISON WITH {self.n_folds}-FOLD CV")
        print("="*50)
        
        self.fetch_data()
        self.engineer_features()
        self.engineer_advanced_features()
        
        return self.run_kfold_cv()
    
    def plot_results(self) -> None:
        """
        Plots detailed results including classifier performance metrics.
        """
        valid_results = {method: results for method, results in self.results.items() 
                       if isinstance(results, dict) and "error" not in results}
        
        if not valid_results:
            print("No valid results to plot. All methods encountered errors.")
            return
        
        try:
            data = []
            baseline_data = []
            for norm_method, model_results in valid_results.items():
                for model_name, metrics in model_results.items():
                    for metric_name, value in metrics.items():
                        if not metric_name.endswith("_std") and not metric_name.endswith("_values"):
                            try:
                                if isinstance(value, (int, float)) and not isinstance(value, bool):
                                    data_point = {
                                        "Normalization": norm_method,
                                        "Model": model_name,
                                        "Metric": metric_name,
                                        "Value": value
                                    }
                                    
                                    data.append(data_point)
                                    
                                    if norm_method == self.baseline_method:
                                        baseline_data.append(data_point)
                            except Exception as e:
                                print(f"Warning: Could not process metric {metric_name} with value {value}: {str(e)}")
            
            if not data:
                print("No data available for plotting. Check if results contain valid metrics.")
                return
                
            df_results = pd.DataFrame(data)
            df_baseline = pd.DataFrame(baseline_data) if baseline_data else None
            
            required_cols = ["Normalization", "Model", "Metric", "Value"]
            if not all(col in df_results.columns for col in required_cols):
                print(f"Warning: DataFrame missing required columns. Columns found: {df_results.columns.tolist()}")
                return
            
            self.plot_classification_metrics(df_results, df_baseline)
            self.plot_feature_distributions()
            self.plot_roc_curves()
            self.plot_confusion_matrices()
            self.plot_fold_comparison()
            
        except Exception as e:
            print(f"Error in plotting results: {str(e)}")
            traceback.print_exc()
    
    def plot_classification_metrics(self, df_results: pd.DataFrame, df_baseline: pd.DataFrame = None) -> None:
        """
        Plots classification metrics for all normalization methods.
        
        Args:
            df_results: DataFrame with all results
            df_baseline: DataFrame with baseline results
        """
        if df_results is None or df_results.empty:
            print("No data available for classification metrics plot")
            return
            
        if not all(col in df_results.columns for col in ["Normalization", "Model", "Metric", "Value"]):
            print(f"Missing required columns for classification metrics. Available columns: {df_results.columns.tolist()}")
            return
        
        classification_metrics = ["accuracy", "f1", "precision", "recall", "roc_auc"]
        available_metrics = df_results["Metric"].unique()
        
        metrics_to_plot = [metric for metric in classification_metrics if metric in available_metrics]
        
        if not metrics_to_plot:
            print(f"No classification metrics found in data. Available metrics: {available_metrics}")
            return
        
        n_metrics = len(metrics_to_plot)
        n_rows = (n_metrics + 1) // 2
        
        fig = plt.figure(figsize=(20, 6 * n_rows))
        
        for i, metric in enumerate(metrics_to_plot):
            try:
                plt.subplot(n_rows, 2, i+1)
                df_metric = df_results[df_results["Metric"] == metric]
                
                if df_metric.empty:
                    continue
                
                plot = sns.barplot(x="Normalization", y="Value", hue="Model", data=df_metric)
                
                if df_baseline is not None:
                    baseline_metric = df_baseline[df_baseline["Metric"] == metric]
                    if not baseline_metric.empty:
                        for model in ["lightgbm", "lstm"]:
                            model_baseline = baseline_metric[baseline_metric["Model"] == model]
                            if not model_baseline.empty:
                                baseline_val = model_baseline["Value"].values
                                if len(baseline_val) > 0:
                                    plt.axhline(
                                        y=baseline_val[0], 
                                        linestyle="--", 
                                        color="red" if model == "lightgbm" else "blue",
                                        alpha=0.7,
                                        label=f"{self.baseline_method} {model} baseline"
                                    )
                
                plt.title(f"{metric.capitalize()} by Normalization Method")
                plt.xlabel("Normalization Method")
                plt.ylabel(metric.capitalize())
                plt.grid(axis="y", linestyle="--", alpha=0.7)
                plt.legend(title="Model")
                plt.xticks(rotation=45)
                
                try:
                    for container in plot.containers:
                        plot.bar_label(container, fmt="%.3f", fontsize=8)
                except Exception as e:
                    print(f"Could not add bar labels: {str(e)}")
            
            except Exception as e:
                print(f"Error plotting {metric}: {str(e)}")
        
        plt.tight_layout()
        try:
            if not os.path.exists("visualizations"):
                os.makedirs("visualizations")
            plt.savefig("visualizations/classification_metrics.png", dpi=300, bbox_inches="tight")
        except Exception as e:
            print(f"Could not save figure: {str(e)}")
        plt.close()

    def plot_feature_distributions(self) -> None:
        """
        Plots feature distributions before and after normalization for selected features.
        Handles missing data gracefully.
        """
        if not self.feature_distributions:
            print("No feature distribution data available.")
            return
        
        methods_to_plot = [self.baseline_method] + [m for m in self.norm_methods if m != self.baseline_method]
        methods_to_plot = [m for m in methods_to_plot if m in self.feature_distributions]
        
        if not methods_to_plot:
            print("No valid normalization methods in feature distributions.")
            return
            
        try:
            if self.features is not None and not self.features.empty:
                feature_cols = list(self.features.columns)[:min(3, len(self.features.columns))]
                
                if not feature_cols:
                    print("No features available for plotting distributions")
                    return
                    
                for feature in feature_cols:
                    feature_exists = True
                    for method in methods_to_plot:
                        if (method not in self.feature_distributions or
                            'before' not in self.feature_distributions[method] or
                            'after' not in self.feature_distributions[method] or
                            feature not in self.feature_distributions[method]['before']['train'].columns or
                            feature not in self.feature_distributions[method]['after']['train'].columns):
                            feature_exists = False
                            break
                    
                    if not feature_exists:
                        print(f"Skipping feature {feature} as it's not available in all distributions")
                        continue
                        
                    plt.figure(figsize=(15, 10))
                    
                    for i, method in enumerate(methods_to_plot):
                        plt.subplot(len(methods_to_plot), 2, i*2 + 1)
                        before_train = self.feature_distributions[method]['before']['train'][feature]
                        sns.histplot(before_train, kde=True)
                        plt.title(f"{method} - Before Normalization")
                        plt.xlabel(feature)
                        
                        plt.subplot(len(methods_to_plot), 2, i*2 + 2)
                        after_train = self.feature_distributions[method]['after']['train'][feature]
                        sns.histplot(after_train, kde=True)
                        plt.title(f"{method} - After Normalization")
                        plt.xlabel(feature)
                    
                    plt.suptitle(f"Distribution of '{feature}' Before and After Normalization")
                    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                    plt.savefig(f"visualizations/feature_distribution_{feature}.png", dpi=300, bbox_inches="tight")
                    plt.close()
                
                if len(feature_cols) > 0:
                    feature = feature_cols[0]
                    
                    feature_exists = True
                    for method in methods_to_plot:
                        if (method not in self.feature_distributions or
                            'after' not in self.feature_distributions[method] or
                            feature not in self.feature_distributions[method]['after']['train'].columns):
                            feature_exists = False
                            break
                    
                    if feature_exists:
                        n_methods = len(methods_to_plot)
                        ncols = min(n_methods, 2)
                        nrows = (n_methods + 1) // 2
                        
                        plt.figure(figsize=(7 * ncols, 5 * nrows))
                        
                        for i, method in enumerate(methods_to_plot):
                            plt.subplot(nrows, ncols, i+1)
                            after_train = self.feature_distributions[method]['after']['train'][feature]
                            sns.histplot(after_train, kde=True)
                            plt.title(f"{method}")
                            plt.xlabel(feature)
                        
                        plt.suptitle(f"Distribution of '{feature}' Across Normalization Methods")
                        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                        plt.savefig(f"visualizations/normalization_comparison_{feature}.png", dpi=300, bbox_inches="tight")
                        plt.close()
                
        except Exception as e:
            print(f"Error plotting feature distributions: {str(e)}")
            traceback.print_exc()
    
    def plot_roc_curves(self) -> None:
        """
        Plots ROC curves for different normalization methods.
        Handles cases where LSTM predictions might not be available or contain NaN values.
        """
        fold_name = "fold_1"
        
        methods_to_plot = [self.baseline_method] + [m for m in self.norm_methods if m != self.baseline_method]
        
        for model_name in ["lightgbm", "lstm"]:
            try:
                plt.figure(figsize=(10, 8))
                curves_plotted = 0
                
                for method in methods_to_plot:
                    if (method in self.fold_results and 
                        fold_name in self.fold_results[method] and
                        model_name in self.fold_results[method][fold_name] and
                        "predictions" in self.fold_results[method][fold_name]):
                        
                        predictions = self.fold_results[method][fold_name]["predictions"]
                        
                        if model_name == "lightgbm":
                            if all(col in predictions.columns for col in ["actual_direction", "lgb_pred_proba"]):
                                y_true = predictions["actual_direction"]
                                y_score = predictions["lgb_pred_proba"]
                                
                                fpr, tpr, _ = roc_curve(y_true, y_score)
                                roc_auc = roc_auc_score(y_true, y_score)
                                
                                plt.plot(fpr, tpr, lw=2, label=f'{method} (AUC = {roc_auc:.3f})')
                                curves_plotted += 1
                        else:
                            if all(col in predictions.columns for col in ["actual_direction", "lstm_pred_proba"]):
                                mask = ~predictions["lstm_pred_proba"].isna()
                                if mask.sum() > 10:
                                    y_true = predictions.loc[mask, "actual_direction"]
                                    y_score = predictions.loc[mask, "lstm_pred_proba"]
                                    
                                    fpr, tpr, _ = roc_curve(y_true, y_score)
                                    roc_auc = roc_auc_score(y_true, y_score)
                                    
                                    plt.plot(fpr, tpr, lw=2, label=f'{method} (AUC = {roc_auc:.3f})')
                                    curves_plotted += 1
                
                if curves_plotted > 0:
                    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                    plt.xlim([0.0, 1.0])
                    plt.ylim([0.0, 1.05])
                    plt.xlabel('False Positive Rate')
                    plt.ylabel('True Positive Rate')
                    plt.title(f'ROC Curves for {model_name.upper()}')
                    plt.legend(loc="lower right")
                    plt.grid(True, alpha=0.3)
                    plt.savefig(f"visualizations/roc_curves_{model_name}.png", dpi=300, bbox_inches="tight")
                else:
                    print(f"No data available to plot ROC curves for {model_name}")
                
                plt.close()
                
            except Exception as e:
                print(f"Error plotting ROC curves for {model_name}: {str(e)}")
                traceback.print_exc()
                plt.close()
    
    def plot_confusion_matrices(self) -> None:
        """
        Plots confusion matrices for different normalization methods.
        Handles missing data and NaN values gracefully.
        """
        fold_name = "fold_1"
        
        methods_to_plot = [self.baseline_method] + [m for m in self.norm_methods if m != self.baseline_method]
        
        for model_name in ["lightgbm", "lstm"]:
            try:
                valid_methods = []
                matrices = []
                
                for method in methods_to_plot:
                    if (method in self.fold_results and 
                        fold_name in self.fold_results[method] and
                        model_name in self.fold_results[method][fold_name] and
                        "confusion_matrix" in self.fold_results[method][fold_name][model_name]):
                        
                        cm = self.fold_results[method][fold_name][model_name]["confusion_matrix"]
                        if np.sum(cm) > 0:
                            valid_methods.append(method)
                            matrices.append(cm)
                
                if valid_methods:
                    n_methods = len(valid_methods)
                    ncols = min(n_methods, 2)
                    nrows = (n_methods + 1) // 2
                    
                    plt.figure(figsize=(7 * ncols, 5 * nrows))
                    
                    for i, (method, cm) in enumerate(zip(valid_methods, matrices)):
                        plt.subplot(nrows, ncols, i+1)
                        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
                        plt.title(f'{method}')
                        plt.ylabel('True Label')
                        plt.xlabel('Predicted Label')
                    
                    plt.suptitle(f'Confusion Matrices for {model_name.upper()}')
                    plt.tight_layout(rect=[0, 0, 1, 0.96])
                    plt.savefig(f"visualizations/confusion_matrices_{model_name}.png", dpi=300, bbox_inches="tight")
                else:
                    print(f"No confusion matrices available for {model_name}")
                
                plt.close()
                
            except Exception as e:
                print(f"Error plotting confusion matrices for {model_name}: {str(e)}")
                plt.close()
    
    def plot_fold_comparison(self) -> None:
        """
        Plot detailed fold-by-fold comparison between baseline and best methods.
        """
        best_methods = {}
        for metric in ["accuracy", "f1", "roc_auc"]:
            for model_name in ["lightgbm", "lstm"]:
                best_value = 0
                best_method = None
                
                for method, results in self.results.items():
                    if (model_name in results and 
                        metric in results[model_name] and 
                        results[model_name][metric] > best_value):
                        best_value = results[model_name][metric]
                        best_method = method
                
                if best_method:
                    key = f"{model_name}_{metric}"
                    best_methods[key] = best_method
        
        for metric in ["accuracy", "f1", "roc_auc"]:
            for model_name in ["lightgbm", "lstm"]:
                key = f"{model_name}_{metric}"
                if key not in best_methods:
                    continue
                    
                best_method = best_methods[key]
                if best_method == self.baseline_method:
                    continue
                
                plt.figure(figsize=(12, 6))
                
                baseline_values = []
                best_values = []
                fold_labels = []
                
                num_folds = len(self.fold_results[self.baseline_method])
                
                for fold in range(num_folds):
                    fold_name = f"fold_{fold+1}"
                    fold_labels.append(f"Fold {fold+1}")
                    
                    if (fold_name in self.fold_results[self.baseline_method] and
                        model_name in self.fold_results[self.baseline_method][fold_name] and
                        metric in self.fold_results[self.baseline_method][fold_name][model_name]):
                        baseline_values.append(
                            self.fold_results[self.baseline_method][fold_name][model_name][metric]
                        )
                    else:
                        baseline_values.append(0)
                    
                    if (fold_name in self.fold_results[best_method] and
                        model_name in self.fold_results[best_method][fold_name] and
                        metric in self.fold_results[best_method][fold_name][model_name]):
                        best_values.append(
                            self.fold_results[best_method][fold_name][model_name][metric]
                        )
                    else:
                        best_values.append(0)
                
                bar_width = 0.35
                index = np.arange(len(fold_labels))
                
                plt.bar(index, baseline_values, bar_width, 
                       label=f"Baseline ({self.baseline_method})", color="blue", alpha=0.7)
                plt.bar(index + bar_width, best_values, bar_width,
                       label=f"Best ({best_method})", color="green", alpha=0.7)
                
                plt.xlabel("Fold")
                plt.ylabel(metric.capitalize())
                plt.title(f"{model_name.upper()} - {metric.capitalize()} Across Folds")
                plt.xticks(index + bar_width / 2, fold_labels)
                plt.legend()
                plt.grid(axis="y", linestyle="--", alpha=0.3)
                
                avg_baseline = np.mean(baseline_values)
                avg_best = np.mean(best_values)
                improvement = ((avg_best - avg_baseline) / avg_baseline) * 100
                
                plt.figtext(
                    0.5, 0.01, 
                    f"Overall improvement: {improvement:.2f}% ({avg_best:.4f} vs {avg_baseline:.4f})",
                    ha="center", fontsize=12
                )
                
                plt.tight_layout(rect=[0, 0.05, 1, 1])
                plt.savefig(f"visualizations/fold_comparison_{model_name}_{metric}.png", dpi=300, bbox_inches="tight")
                plt.close()
    
    def print_summary(self) -> None:
        """
        Print detailed summary of cross-validation results.
        """
        print("\n" + "="*60)
        print(f"NORMALIZATION METHOD PERFORMANCE SUMMARY ({self.n_folds}-FOLD CV)")
        print("="*60)
        
        best_methods = {}
        for model_name in ["lightgbm", "lstm"]:
            best_methods[model_name] = {}
            for metric in ["accuracy", "f1", "roc_auc"]:
                best_val = 0
                best_method = None
                
                for method, results in self.results.items():
                    if (model_name in results and 
                        metric in results[model_name] and 
                        results[model_name][metric] > best_val):
                        best_val = results[model_name][metric]
                        best_method = method
                
                if best_method:
                    best_methods[model_name][metric] = (best_method, best_val)
        
        for model_name in ["lightgbm", "lstm"]:
            print(f"\n{model_name.upper()} MODEL RESULTS:")
            print("-" * 40)
            
            for metric in ["accuracy", "f1", "roc_auc"]:
                if metric in best_methods[model_name]:
                    best_method, best_val = best_methods[model_name][metric]
                    print(f"Best {metric}: {best_method} ({best_val:.4f})")
                    
                    if (self.baseline_method in self.results and 
                        model_name in self.results[self.baseline_method] and
                        metric in self.results[self.baseline_method][model_name]):
                        
                        baseline_val = self.results[self.baseline_method][model_name][metric]
                        diff = best_val - baseline_val
                        pct_change = (diff / baseline_val) * 100 if baseline_val != 0 else float("inf")
                        
                        print(f"  vs. baseline ({self.baseline_method}): {baseline_val:.4f} ({pct_change:+.2f}%)")
                        
                        if f"{metric}_std" in self.results[best_method][model_name]:
                            best_std = self.results[best_method][model_name][f"{metric}_std"]
                            baseline_std = self.results[self.baseline_method][model_name][f"{metric}_std"]
                            
                            print(f"  Std Dev: {best_std:.4f} vs. baseline: {baseline_std:.4f}")
            
            print("\nAll methods:")
            rows = []
            for method in self.norm_methods:
                if method not in self.results:
                    continue
                    
                if model_name not in self.results[method]:
                    continue
                
                row = [method]
                
                for metric in ["accuracy", "f1", "roc_auc"]:
                    if metric in self.results[method][model_name]:
                        val = self.results[method][model_name][metric]
                        std = self.results[method][model_name].get(f"{metric}_std", 0)
                        row.append(f"{val:.4f} ± {std:.4f}")
                    else:
                        row.append("N/A")
                
                rows.append(row)
            
            if rows:
                headers = ["Method", "Accuracy", "F1-Score", "ROC AUC"]
                row_format = "{:<15} {:<20} {:<20} {:<20}"
                print(row_format.format(*headers))
                print("-" * 75)
                for row in rows:
                    print(row_format.format(*row))
        
        print("\n" + "="*40)
        print("OVERALL BEST NORMALIZATION METHODS")
        print("="*40)
        
        for model_name in ["lightgbm", "lstm"]:
            print(f"\n{model_name.upper()}:")
            
            method_ranks = {}
            for method in self.norm_methods:
                if method not in self.results or model_name not in self.results[method]:
                    continue
                
                ranks = []
                for metric in ["accuracy", "f1", "roc_auc"]:
                    if metric not in self.results[method][model_name]:
                        continue
                        
                    values = [(m, self.results[m][model_name][metric]) 
                             for m in self.results 
                             if model_name in self.results[m] and metric in self.results[m][model_name]]
                    
                    values.sort(key=lambda x: x[1], reverse=True)
                    
                    for i, (m, _) in enumerate(values):
                        if m == method:
                            ranks.append(i + 1)
                            break
                
                if ranks:
                    method_ranks[method] = sum(ranks) / len(ranks)
            
            sorted_methods = sorted(method_ranks.items(), key=lambda x: x[1])
            
            print("Top methods by average rank:")
            for i, (method, avg_rank) in enumerate(sorted_methods[:3], 1):
                acc = self.results[method][model_name].get("accuracy", 0)
                f1 = self.results[method][model_name].get("f1", 0)
                roc = self.results[method][model_name].get("roc_auc", 0)
                
                print(f"{i}. {method} (avg rank: {avg_rank:.2f}, acc: {acc:.4f}, f1: {f1:.4f}, roc_auc: {roc:.4f})")
        
        # print("\n" + "="*40)
        # print("CONCLUSION AND RECOMMENDATIONS")
        # print("="*40)
        
        # all_ranks = {}
        # for method in self.norm_methods:
        #     if method not in self.results:
        #         continue
                
        #     ranks = []
        #     for model_name in ["lightgbm", "lstm"]:
        #         if model_name not in self.results[method]:
        #             continue
                    
        #         for metric in ["accuracy", "f1", "roc_auc"]:
        #             if metric not in self.results[method][model_name]:
        #                 continue
                        
        #             values = [(m, self.results[m][model_name][metric]) 
        #                      for m in self.results 
        #                      if model_name in self.results[m] and metric in self.results[m][model_name]]
                    
        #             values.sort(key=lambda x: x[1], reverse=True)
                    
        #             for i, (m, _) in enumerate(values):
        #                 if m == method:
        #                     ranks.append(i + 1)
        #                     break
            
        #     if ranks:
        #         all_ranks[method] = sum(ranks) / len(ranks)
        
        # sorted_all_methods = sorted(all_ranks.items(), key=lambda x: x[1])
        
        # if sorted_all_methods:
        #     best_overall = sorted_all_methods[0][0]
        #     print(f"Based on the cross-validation results, the best overall normalization method is: {best_overall}")
        #     print("\nRecommendations:")
        #     print(f"1. Consider using {best_overall} normalization for Norwegian stock market prediction tasks")
        #     print(f"2. The {best_overall} method showed consistent performance across different models and metrics")
        #     print(f"3. For LightGBM specifically, consider {best_methods['lightgbm'].get('accuracy', ('N/A', 0))[0]} for classification accuracy")
            
        #     print("\nNote: These recommendations are based on historical testing of the Norwegian stock market and should be")
        #     print("periodically validated as market conditions change.")

    def perform_statistical_tests(self) -> Dict:
        """
        Performs statistical significance tests comparing normalization methods against the baseline.
        Uses Wilcoxon signed-rank tests on fold results.
        
        Returns:
            Dict: Statistical test results with p-values
        """
        if not self.fold_results:
            print("No fold results available for statistical testing.")
            return {}
        
        print("\nPerforming statistical significance tests (Wilcoxon)...")
        
        stat_results = {}
        
        metrics_to_test = ["accuracy", "f1", "roc_auc"]
        models_to_test = ["lightgbm"]
        
        for method in self.fold_results:
            for fold in self.fold_results[method]:
                if "lstm" in self.fold_results[method][fold]:
                    models_to_test.append("lstm")
                    break
            break
        
        for model_name in models_to_test:
            stat_results[model_name] = {}
            
            for metric in metrics_to_test:
                stat_results[model_name][metric] = {}
                
                baseline_values = []
                for fold in range(self.n_folds):
                    fold_name = f"fold_{fold+1}"
                    if (fold_name in self.fold_results[self.baseline_method] and
                        model_name in self.fold_results[self.baseline_method][fold_name] and
                        metric in self.fold_results[self.baseline_method][fold_name][model_name]):
                        baseline_values.append(self.fold_results[self.baseline_method][fold_name][model_name][metric])
                
                if not baseline_values:
                    continue
                    
                for method in self.norm_methods:
                    if method == self.baseline_method:
                        continue
                    
                    method_values = []
                    for fold in range(self.n_folds):
                        fold_name = f"fold_{fold+1}"
                        if (fold_name in self.fold_results[method] and
                            model_name in self.fold_results[method][fold_name] and
                            metric in self.fold_results[method][fold_name][model_name]):
                            method_values.append(self.fold_results[method][fold_name][model_name][metric])
                    
                    if not method_values:
                        continue
                    
                    min_length = min(len(baseline_values), len(method_values))
                    if min_length < 2:
                        print(f"Warning: Not enough samples for {model_name}/{metric}/{method} - need at least 2.")
                        continue
                    
                    baseline_values_trimmed = baseline_values[:min_length]
                    method_values_trimmed = method_values[:min_length]
                    
                    mean_diff = np.mean(method_values_trimmed) - np.mean(baseline_values_trimmed)
                    
                    try:
                        w_stat, p_value_w = stats.wilcoxon(method_values_trimmed, baseline_values_trimmed)
                    except Exception as e:
                        print(f"Error in Wilcoxon test for {model_name}/{metric}/{method}: {str(e)}")
                        w_stat, p_value_w = np.nan, np.nan

                    is_significant = p_value_w < 0.05 if not np.isnan(p_value_w) else False
                    
                    stat_results[model_name][metric][method] = {
                        "mean_diff": mean_diff,
                        "percent_improvement": (mean_diff / np.mean(baseline_values_trimmed)) * 100,
                        "w_stat": w_stat,
                        "p_value": p_value_w,
                        "significant": is_significant,
                        "baseline_mean": np.mean(baseline_values_trimmed),
                        "method_mean": np.mean(method_values_trimmed)
                    }
        
        self.statistical_test_results = stat_results
        return stat_results

    def print_statistical_summary(self) -> None:
        """
        Prints a summary of statistical significance test results.
        """
        print("\n" + "="*70)
        print("STATISTICAL SIGNIFICANCE ANALYSIS (WILCOXON TEST)")
        print("="*70)
        
        for model_name in self.statistical_test_results:
            print(f"\n{model_name.upper()} MODEL RESULTS:")
            print("-" * 50)
            
            for metric in self.statistical_test_results[model_name]:
                print(f"\n{metric.upper()} (baseline: {self.baseline_method}):")
                
                sorted_methods = sorted(
                    self.statistical_test_results[model_name][metric].items(),
                    key=lambda x: x[1]["mean_diff"],
                    reverse=True
                )
                
                headers = ["Method", "Mean Diff", "% Improv", "p-value", "Significant"]
                print(f"{headers[0]:<15} {headers[1]:<12} {headers[2]:<10} {headers[3]:<12} {headers[4]:<10}")
                print("-" * 60)
                
                for method, results in sorted_methods:
                    mean_diff = f"{results['mean_diff']:.4f}"
                    pct_improv = f"{results['percent_improvement']:.2f}%"
                    p_value = f"{results['p_value']:.4f}" if not np.isnan(results['p_value']) else "N/A"
                    sig = "Yes*" if results['significant'] else "No"
                    
                    if results['significant']:
                        p_value += "*"
                    
                    print(f"{method:<15} {mean_diff:<12} {pct_improv:<10} {p_value:<12} {sig:<10}")
                
                print("\n* p < 0.05 indicates statistical significance")
        
        print("\n" + "="*50)
        print("SUMMARY OF STATISTICAL FINDINGS")
        print("="*50)
        
        significant_findings = []
        
        for model_name in self.statistical_test_results:
            for metric in self.statistical_test_results[model_name]:
                sig_methods = [
                    (method, results) 
                    for method, results in self.statistical_test_results[model_name][metric].items()
                    if results['significant'] and results['mean_diff'] > 0
                ]
                
                sig_methods.sort(key=lambda x: x[1]['mean_diff'], reverse=True)
                
                if sig_methods:
                    top_method, top_results = sig_methods[0]
                    finding = f"{top_method} showed statistically significant improvement over {self.baseline_method} "
                    finding += f"for {metric} in {model_name} (p < 0.05, {top_results['percent_improvement']:.2f}% improvement)"
                    significant_findings.append(finding)
        
        if significant_findings:
            print("\nKey significant findings:")
            for i, finding in enumerate(significant_findings, 1):
                print(f"{i}. {finding}")
        else:
            print("\nNo statistically significant improvements were found over the baseline method.")
        
        print("\nNote: Statistical significance was tested using the Wilcoxon signed-rank test.")

    def plot_statistical_significance(self, save_path="visualizations/") -> None:
        """
        Creates visualizations highlighting statistical significance of results.
        
        Args:
            save_path: Directory to save visualizations
        """
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        print(f"\nGenerating statistical significance visualizations in {save_path}")
        
        for model_name in self.statistical_test_results:
            for metric in self.statistical_test_results[model_name]:
                try:
                    methods = []
                    mean_diffs = []
                    is_significant = []
                    
                    for method, results in self.statistical_test_results[model_name][metric].items():
                        methods.append(method)
                        mean_diffs.append(results['mean_diff'])
                        is_significant.append(results['significant'])
                    
                    if not methods:
                        continue
                    
                    sorted_indices = np.argsort(mean_diffs)[::-1]  # Descending
                    methods = [methods[i] for i in sorted_indices]
                    mean_diffs = [mean_diffs[i] for i in sorted_indices]
                    is_significant = [is_significant[i] for i in sorted_indices]
                    
                    plt.figure(figsize=(12, 6))
                    
                    colors = ['green' if sig else 'gray' for sig in is_significant]
                    
                    bars = plt.bar(range(len(methods)), mean_diffs, color=colors, alpha=0.7)
                    
                    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
                    
                    for i, bar in enumerate(bars):
                        height = bar.get_height()
                        if is_significant[i]:
                            plt.text(
                                bar.get_x() + bar.get_width()/2,
                                height + 0.001 if height > 0 else height - 0.003,
                                '*',
                                ha='center',
                                va='bottom' if height > 0 else 'top',
                                fontsize=16,
                                fontweight='bold'
                            )
                    
                    plt.title(f'Mean Difference vs {self.baseline_method} - {metric} ({model_name})', fontsize=14)
                    plt.xlabel('Normalization Method')
                    plt.ylabel(f'Mean Difference in {metric}')
                    plt.xticks(range(len(methods)), methods, rotation=45)
                    plt.grid(axis='y', linestyle='--', alpha=0.3)

                    legend_elements = [
                        Patch(facecolor='green', alpha=0.7, label='Statistically Significant (p < 0.05)'),
                        Patch(facecolor='gray', alpha=0.7, label='Not Significant')
                    ]
                    plt.legend(handles=legend_elements, loc='best')
                    
                    plt.tight_layout()
                    plt.savefig(f"{save_path}significance_{model_name}_{metric}.png", dpi=300, bbox_inches="tight")
                    plt.close()
                    
                except Exception as e:
                    print(f"Error plotting statistical significance for {model_name}/{metric}: {str(e)}")
                    plt.close()

    def plot_pvalue_heatmap(self, save_path="visualizations/") -> None:
        """
        Creates a heatmap visualization of p-values across methods and metrics.
        
        Args:
            save_path: Directory to save visualizations
        """
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        for model_name in self.statistical_test_results:
            try:
                metrics = list(self.statistical_test_results[model_name].keys())
                
                if not metrics:
                    continue
                    
                all_methods = set()
                for metric in metrics:
                    all_methods.update(self.statistical_test_results[model_name][metric].keys())
                all_methods = sorted(list(all_methods))
                
                if not all_methods:
                    continue
                
                heatmap_data = np.zeros((len(metrics), len(all_methods)))
                annot_data = np.empty((len(metrics), len(all_methods)), dtype=object)
                
                for i, metric in enumerate(metrics):
                    for j, method in enumerate(all_methods):
                        if method in self.statistical_test_results[model_name][metric]:
                            p_value = self.statistical_test_results[model_name][metric][method]['p_value']
                            
                            p_value = p_value if not np.isnan(p_value) else 1.0
                            
                            heatmap_data[i, j] = p_value
                            
                            annot_data[i, j] = f"{p_value:.3f}"
                            if p_value < 0.05:
                                annot_data[i, j] += "*"
                            if p_value < 0.01:
                                annot_data[i, j] += "*"
                        else:
                            heatmap_data[i, j] = 1.0  # Max p-value for missing data
                            annot_data[i, j] = "N/A"
                
                plt.figure(figsize=(12, max(6, len(metrics) * 0.8)))
                
                cmap = plt.cm.YlGnBu_r
                
                ax = sns.heatmap(
                    heatmap_data,
                    annot=annot_data,
                    fmt="",
                    cmap=cmap,
                    vmin=0,
                    vmax=0.1,
                    linewidths=0.5,
                    cbar_kws={'label': 'p-value'}
                )
                
                cbar = ax.collections[0].colorbar
                cbar.set_ticks([0, 0.01, 0.05, 0.1])
                cbar.set_ticklabels(['0', '0.01', '0.05', '≥0.1'])
                
                plt.title(f'P-values Heatmap for {model_name}', fontsize=14)
                plt.yticks(np.arange(len(metrics)) + 0.5, metrics, rotation=0)
                plt.xticks(np.arange(len(all_methods)) + 0.5, all_methods, rotation=45, ha='right')
                
                plt.tight_layout()
                plt.savefig(f"{save_path}pvalue_heatmap_{model_name}.png", dpi=300, bbox_inches="tight")
                plt.close()
                
            except Exception as e:
                print(f"Error creating p-value heatmap for {model_name}: {str(e)}")
                plt.close()
    
    def save_results_to_csv(self, filename="normalization_results.csv"):
        """
        Save the results to a CSV file for further analysis.
        
        Args:
            filename: Name of the CSV file
        """
        rows = []
        
        for method in self.results:
            for model in self.results[method]:
                row = {"Normalization": method, "Model": model}
                
                for metric, value in self.results[method][model].items():
                    if not metric.endswith("_std") and not metric.endswith("_values"):
                        row[metric] = value
                        if f"{metric}_std" in self.results[method][model]:
                            row[f"{metric}_std"] = self.results[method][model][f"{metric}_std"]
                
                rows.append(row)
        
        df = pd.DataFrame(rows)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")

In [ ]:
print("\n" + "="*60)
print("Impact of Normalization Techniques with Statistical Significance Testing")
print("="*60)
norm_methods = [
    "min_max",
    "z_score",
    "median",
    "sigmoid",
    "tanh_estimator"
]
baseline = "min_max"
fast_mode = False
n_folds = 10
time_series_split = True
gap = 5

research = NormalizationResearch(
    cache_path="data",
    fast_mode=fast_mode,
    start_date="2014-01-01",
    end_date="2024-12-31",
    norm_methods=norm_methods,
    baseline_method=baseline,
    market_index="OSEBX.OL",
    n_folds=n_folds,
    time_series_split=time_series_split,
    gap=gap,
    lookback_window=10,
    random_seed=2025
)

results = research.run_normalization_comparison()

research.perform_statistical_tests()
research.print_statistical_summary()
research.plot_statistical_significance()
research.plot_pvalue_heatmap()

research.print_summary()
research.save_results_to_csv("results/normalization_results_osebx_with_stats.csv")
research.plot_results()


Impact of Normalization Techniques with Statistical Significance Testing
Using device: cuda

NORMALIZATION TECHNIQUES COMPARISON WITH 10-FOLD CV
Fetching OSEBX.OL data from 2014-01-01 to 2024-12-31...
Data already downloaded. Loading from file...
Engineering features (time-aware approach)...
Features prepared: 1709 samples with 19 features
Class distribution: {1: 0.5792861322410766, 0: 0.42071386775892333}
Adding advanced features (time-aware approach)...


[I 2025-03-20 03:03:18,694] A new study created in memory with name: no-name-8a783415-d8d9-4ff0-af89-efd4600ecb05
[I 2025-03-20 03:03:18,779] Trial 0 finished with value: 0.7378460000200813 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.013661178428178952, 'n_estimators': 456, 'num_leaves': 48, 'max_depth': 6, 'subsample': 0.7552942184455931, 'colsample_bytree': 0.7030385741204636, 'reg_alpha': 1.3087232248047334, 'reg_lambda': 0.6869757398160233, 'min_child_samples': 49}. Best is trial 0 with value: 0.7378460000200813.
[I 2025-03-20 03:03:18,827] Trial 1 finished with value: 0.7430340219972068 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.06323892443630524, 'n_estimators': 282, 'num_leaves': 42, 'max_depth': 3, 'subsample': 0.9077831487751247, 'colsample_bytree': 0.6012684467021177, 'reg_alpha': 0.31439409038438626, 'reg_lambda': 1.0912521853098582, 'min_child_samples': 47}. Best is trial 1 with value: 0.7430340219972068.
[I 2025-03-20 03:03:18,877] Trial 


Running 10-fold cross-validation
Creating 10 time series splits with gap=5
Fold 1: train=153, test=155, gap=5 days
Fold 2: train=308, test=155, gap=5 days
Fold 3: train=463, test=155, gap=5 days
Fold 4: train=618, test=155, gap=5 days
Fold 5: train=773, test=155, gap=5 days
Fold 6: train=928, test=155, gap=5 days
Fold 7: train=1083, test=155, gap=5 days
Fold 8: train=1238, test=155, gap=5 days
Fold 9: train=1393, test=155, gap=5 days
Fold 10: train=1548, test=155, gap=5 days

Optimizing LightGBM and LSTM parameters for baseline method...

Optimizing LightGBM for fold 1 using min_max normalization
Optimizing LightGBM hyperparameters with CV...


[I 2025-03-20 03:03:18,925] Trial 3 finished with value: 0.794725944849784 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.03308671790989007, 'n_estimators': 162, 'num_leaves': 10, 'max_depth': 5, 'subsample': 0.9963593754983149, 'colsample_bytree': 0.805256515122397, 'reg_alpha': 3.0841806167907238, 'reg_lambda': 0.1301677362213169, 'min_child_samples': 21}. Best is trial 3 with value: 0.794725944849784.
[I 2025-03-20 03:03:18,999] Trial 4 finished with value: 0.7910122485088833 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.029437387715603364, 'n_estimators': 405, 'num_leaves': 47, 'max_depth': 6, 'subsample': 0.9716350708762611, 'colsample_bytree': 0.7998448041733087, 'reg_alpha': 2.304933318498776, 'reg_lambda': 3.3362817514757888, 'min_child_samples': 29}. Best is trial 3 with value: 0.794725944849784.
[I 2025-03-20 03:03:19,046] Trial 5 finished with value: 0.7843851075339163 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.07045793365915588


Best LightGBM Parameters:
  boosting_type: gbdt
  learning_rate: 0.0819032584475924
  n_estimators: 500
  num_leaves: 17
  max_depth: 10
  subsample: 0.8037845297098607
  colsample_bytree: 0.6626220662335015
  reg_alpha: 0.3014158656716065
  reg_lambda: 1.633537839062495
  min_child_samples: 12

Optimizing LSTM for fold 1 using min_max normalization
Created LSTM sequences with lookback=10: 143 train, 145 test
Optimizing LSTM hyperparameters with CV...


...

[I 2025-03-20 03:19:35,206] Trial 0 finished with value: 0.7360190119021798 and parameters: {'hidden_dim': 45, 'dropout': 0.4551406810921512, 'learning_rate': 0.007331811314274292, 'batch_size': 16, 'weight_decay': 2.064121088592392e-05}. Best is trial 0 with value: 0.7360190119021798.
[I 2025-03-20 03:19:41,518] Trial 1 finished with value: 0.7602867078276914 and parameters: {'hidden_dim': 79, 'dropout': 0.48569536770002286, 'learning_rate': 0.003999161563860724, 'batch_size': 32, 'weight_decay': 3.45872269711161e-05}. Best is trial 1 with value: 0.7602867078276914.
[I 2025-03-20 03:19:56,473] Trial 2 finished with value: 0.8122610101743096 and parameters: {'hidden_dim': 32, 'dropout': 0.21712375681979076, 'learning_rate': 0.0016665895921660098, 'batch_size': 16, 'weight_decay': 2.1517123363927774e-05}. Best is trial 2 with value: 0.8122610101743096.
[I 2025-03-20 03:20:02,728] Trial 3 finished with value: 0.6639936725280756 and parameters: {'hidden_dim': 127, 'dropout': 0.28730816242


Best LSTM Parameters:
  hidden_dim: 46
  dropout: 0.20823070018643636
  learning_rate: 0.002921602795527635
  batch_size: 32
  weight_decay: 1.6383383134508966e-06

Optimizing LightGBM for fold 4 using min_max normalization
Optimizing LightGBM hyperparameters with CV...


[I 2025-03-20 03:34:29,650] Trial 0 finished with value: 0.7079161247417418 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.013661178428178952, 'n_estimators': 456, 'num_leaves': 48, 'max_depth': 6, 'subsample': 0.7552942184455931, 'colsample_bytree': 0.7030385741204636, 'reg_alpha': 1.3087232248047334, 'reg_lambda': 0.6869757398160233, 'min_child_samples': 49}. Best is trial 0 with value: 0.7079161247417418.
[I 2025-03-20 03:34:29,744] Trial 1 finished with value: 0.7248946429641542 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.06323892443630524, 'n_estimators': 282, 'num_leaves': 42, 'max_depth': 3, 'subsample': 0.9077831487751247, 'colsample_bytree': 0.6012684467021177, 'reg_alpha': 0.31439409038438626, 'reg_lambda': 1.0912521853098582, 'min_child_samples': 47}. Best is trial 1 with value: 0.7248946429641542.
[I 2025-03-20 03:34:29,895] Trial 2 finished with value: 0.7465996476669382 and parameters: {'boosting_type': 'gbdt', 'learning_rate': 0.01995790746


Best LightGBM Parameters:
  boosting_type: gbdt
  learning_rate: 0.08161162970270688
  n_estimators: 420
  num_leaves: 29
  max_depth: 9
  subsample: 0.7927640746343508
  colsample_bytree: 0.8286362360751653
  reg_alpha: 0.10523475272795739
  reg_lambda: 0.9097489246635381
  min_child_samples: 14

Optimizing LSTM for fold 4 using min_max normalization
Created LSTM sequences with lookback=10: 608 train, 145 test
Optimizing LSTM hyperparameters with CV...


[I 2025-03-20 03:35:03,477] Trial 0 finished with value: 0.5754726009718998 and parameters: {'hidden_dim': 45, 'dropout': 0.4551406810921512, 'learning_rate': 0.007331811314274292, 'batch_size': 16, 'weight_decay': 2.064121088592392e-05}. Best is trial 0 with value: 0.5754726009718998.
[I 2025-03-20 03:35:15,493] Trial 1 finished with value: 0.6835778163308509 and parameters: {'hidden_dim': 79, 'dropout': 0.48569536770002286, 'learning_rate': 0.003999161563860724, 'batch_size': 32, 'weight_decay': 3.45872269711161e-05}. Best is trial 1 with value: 0.6835778163308509.
[I 2025-03-20 03:35:31,801] Trial 2 finished with value: 0.7121915131941408 and parameters: {'hidden_dim': 32, 'dropout': 0.21712375681979076, 'learning_rate': 0.0016665895921660098, 'batch_size': 16, 'weight_decay': 2.1517123363927774e-05}. Best is trial 2 with value: 0.7121915131941408.
[I 2025-03-20 03:35:40,215] Trial 3 finished with value: 0.6373089601388842 and parameters: {'hidden_dim': 127, 'dropout': 0.28730816242